# Flexible Collective Wisdom — SNR Exploration Notebook

**Purpose.** Compute and compare alternative Signal-to-Noise Ratio (SNR) definitions for a covert-attention dataset, then select one SNR for downstream accuracy/ROC analyses and human-vs-LLM comparisons.

**Expected columns:** `stimID, condition (50_50|80_20|100_0), response (1–6), side_selected (1|2), cue_points (1|2), line1_angle (deg), line2_angle (deg), valid_cue (0|1), TP (0|1), participantID`.

**Workflow overview**
1. Load & preprocess
2. Estimate noise scale (σ) from TP=0 per participant×side
3. Compute SNR candidates (trial-level & condition-level)
4. Diagnostics and cross-validated predictive validity
5. Plots & selection
6. Export winner recipe


In [ ]:
# === Imports ===
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict, Tuple
from dataclasses import dataclass

# Modeling
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, log_loss, brier_score_loss
import statsmodels.api as sm

# Plotting
import matplotlib.pyplot as plt

# Display options
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 160)

# === Config ===
DATA_PATH = Path('data/human_trials.csv')  # <-- change to your path
SAVE_DIR = Path('outputs/snr_eval')
SAVE_DIR.mkdir(parents=True, exist_ok=True)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)


## 1. Load & Preprocess

In [ ]:
def load_data(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    expected_cols = ['stimID','condition','response','side_selected','cue_points',
                     'line1_angle','line2_angle','valid_cue','TP','participantID']
    missing = [c for c in expected_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns: {missing}")
    df['yes'] = (df['response'] >= 4).astype(int)
    df['correct'] = (df['yes'] == df['TP']).astype(int)
    df['participantID'] = df['participantID'].astype(str)
    df['condition'] = df['condition'].astype(str)
    for c in ['side_selected','cue_points','valid_cue','TP']:
        df[c] = df[c].astype(int)
    for c in ['line1_angle','line2_angle']:
        df[c] = df[c].astype(float)
    return df

df = load_data(DATA_PATH)
print(df.head(3))
print(df.condition.value_counts(dropna=False))
print('N trials:', len(df), 'N participants:', df.participantID.nunique())


## 2. Estimate noise (σ) from TP=0 per participant×side

In [ ]:
def robust_sigma(x: np.ndarray) -> float:
    mad = np.median(np.abs(x - np.median(x)))
    if mad == 0:
        s = np.std(x, ddof=1) if x.size > 1 else 1.0
        return float(s if s > 0 else 1.0)
    return float(1.4826 * mad)

def estimate_sigma(df: pd.DataFrame, method: str = 'MAD') -> pd.DataFrame:
    rows = []
    for pid, gpid in df[df['TP']==0].groupby('participantID'):
        for side in [1,2]:
            ang = gpid['line1_angle'] if side==1 else gpid['line2_angle']
            ang = ang.to_numpy()
            if method.upper() == 'MAD':
                sigma = robust_sigma(ang)
            else:
                sigma = float(np.std(ang, ddof=1)) if ang.size>1 else 1.0
                if sigma <= 0: sigma = 1.0
            rows.append({'participantID': pid, 'side': side, 'sigma': sigma, 'n0': int(len(ang))})
    return pd.DataFrame(rows)

sigma_tbl = estimate_sigma(df, method='MAD')
print(sigma_tbl.head())


## 3. Target tilt per trial

In [ ]:
def target_tilt(row) -> float:
    if row['TP'] == 1:
        target_side = row['cue_points'] if row['valid_cue']==1 else (1 if row['cue_points']==2 else 2)
        tilt = abs(row['line1_angle']) if target_side==1 else abs(row['line2_angle'])
        return float(tilt)
    else:
        return 0.0

df['target_tilt_abs'] = df.apply(target_tilt, axis=1)


## 4. Compute SNR candidates

In [ ]:
def attach_sigma(df: pd.DataFrame, sigma_tbl: pd.DataFrame) -> pd.DataFrame:
    sig1 = sigma_tbl[sigma_tbl['side']==1].rename(columns={'sigma':'sigma1','n0':'n01'})[['participantID','sigma1','n01']]
    sig2 = sigma_tbl[sigma_tbl['side']==2].rename(columns={'sigma':'sigma2','n0':'n02'})[['participantID','sigma2','n02']]
    out = df.merge(sig1, on='participantID', how='left').merge(sig2, on='participantID', how='left')
    for c in ['sigma1','sigma2']:
        out[c] = out[c].fillna(out[c].median())
        out[c] = out[c].replace(0, out[c].median())
    return out

df = attach_sigma(df, sigma_tbl)

# Candidate 1: Angle-based
df['SNR_angle'] = df['target_tilt_abs'] / np.where(df['cue_points']==1, df['sigma1'], df['sigma2'])

# Candidate 2: Side-max
df['SNR_side_max'] = np.maximum(abs(df['line1_angle']), abs(df['line2_angle'])) / df[['sigma1','sigma2']].mean(axis=1)

# Candidate 3: Cue-aware weighted
w = np.where(df['valid_cue']==1, 1.0, 0.5)
tilt_cued = np.where(df['cue_points']==1, abs(df['line1_angle']), abs(df['line2_angle']))
df['SNR_cue_weighted'] = (w * tilt_cued) / np.where(df['cue_points']==1, df['sigma1'], df['sigma2'])

# Candidate 4: Condition ordinal proxy
cond_map = {'50_50': 1, '80_20': 2, '100_0': 3}
df['SNR_cond_ord'] = df['condition'].map(cond_map).astype(float)

print(df[['SNR_angle','SNR_side_max','SNR_cue_weighted','SNR_cond_ord']].head())


## 5. Empirical SNR per participant×condition (d′) and hybrid

In [ ]:
def dprime_from_rates(H, F):
    # Loglinear correction to avoid inf/NaN
    eps = 1e-6
    Hc = min(max(H, eps), 1-eps)
    Fc = min(max(F, eps), 1-eps)
    from math import sqrt
    from scipy.stats import norm
    z = norm.ppf
    return float(z(Hc) - z(Fc))

def empirical_dprime_table(df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for (pid, cond), g in df.groupby(['participantID','condition']):
        if (g['TP']==1).any() and (g['TP']==0).any():
            H = g.loc[g['TP']==1, 'yes'].mean()
            F = g.loc[g['TP']==0, 'yes'].mean()
            try:
                dp = dprime_from_rates(H, F)
            except Exception:
                dp = np.nan
        else:
            dp = np.nan
        rows.append({'participantID': pid, 'condition': cond, 'dprime_emp': dp})
    return pd.DataFrame(rows)

try:
    dprime_tbl = empirical_dprime_table(df)
    df = df.merge(dprime_tbl, on=['participantID','condition'], how='left')
    df['SNR_hybrid_emp'] = df['dprime_emp']
except Exception as e:
    print('Empirical d\u2032 step skipped (install scipy). Error:', e)
    df['dprime_emp'] = np.nan
    df['SNR_hybrid_emp'] = np.nan

print(df[['participantID','condition','dprime_emp']].drop_duplicates().head())


## 6. Optional: within-participant z-scoring of SNRs

In [ ]:
SNR_COLS = ['SNR_angle','SNR_side_max','SNR_cue_weighted','SNR_cond_ord','SNR_hybrid_emp']

def zscore_within(df, cols, by='participantID'):
    out = df.copy()
    for c in cols:
        def _z(x):
            s = x.std(ddof=1)
            return (x - x.mean()) / (s if s>0 else 1.0)
        out[c + '_z'] = out.groupby(by)[c].transform(_z)
    return out

dfz = zscore_within(df, SNR_COLS)
print(dfz[[c for c in dfz.columns if c.endswith('_z')]].head())


## 7. Diagnostics: monotonicity, separation, calibration

In [ ]:
def bin_and_acc(df: pd.DataFrame, col: str, n_bins: int = 5) -> pd.DataFrame:
    g = df.copy()
    g['bin'] = pd.qcut(g[col], q=n_bins, duplicates='drop')
    out = g.groupby('bin').agg(acc=('correct','mean'),
                               yes_rate=('yes','mean'),
                               n=('correct','size'),
                               snr_mean=(col,'mean')).reset_index()
    return out

def separation_stats(df: pd.DataFrame, col: str) -> Dict[str,float]:
    from scipy.stats import ks_2samp
    a = df.loc[df['TP']==1, col].dropna()
    b = df.loc[df['TP']==0, col].dropna()
    if len(a)<2 or len(b)<2:
        return {'cohen_d': np.nan, 'ks': np.nan}
    d = (a.mean()-b.mean()) / np.sqrt(0.5*(a.var(ddof=1)+b.var(ddof=1)))
    ks = ks_2samp(a, b).statistic
    return {'cohen_d': float(d), 'ks': float(ks)}

def plot_accuracy_vs_snr(df: pd.DataFrame, col: str, n_bins: int = 5):
    b = bin_and_acc(df, col, n_bins=n_bins)
    plt.figure()
    plt.errorbar(b['snr_mean'], b['acc'], yerr=np.sqrt(b['acc']*(1-b['acc'])/b['n']), fmt='o-')
    plt.xlabel(col)
    plt.ylabel('Proportion correct')
    plt.title(f'Accuracy vs {col} (bin means \u00B1 SE)')
    plt.tight_layout()
    plt.show()

def plot_density_tp(df: pd.DataFrame, col: str):
    plt.figure()
    df.loc[df['TP']==1, col].dropna().plot(kind='hist', bins=30, alpha=0.5, label='TP=1')
    df.loc[df['TP']==0, col].dropna().plot(kind='hist', bins=30, alpha=0.5, label='TP=0')
    plt.legend()
    plt.xlabel(col)
    plt.title(f'Density overlay by TP for {col}')
    plt.tight_layout()
    plt.show()


## 8. Predictive validity (5× CV stratified by participant)

In [ ]:
from typing import List

def cv_metrics_for_snr(df: pd.DataFrame, col: str, k: int = 5) -> Dict[str,float]:
    X = df[[col,'valid_cue']].copy()
    X['condition'] = df['condition']
    X['participantID'] = df['participantID']
    y = df['correct'].astype(int).values

    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=RANDOM_SEED)
    part_labels = df['participantID'].astype('category').cat.codes.values

    aucs, lls, brs = [], [], []
    for tr, te in skf.split(np.zeros_like(part_labels), part_labels):
        Xtr, Xte = X.iloc[tr], X.iloc[te]
        ytr, yte = y[tr], y[te]

        num = [col, 'valid_cue']
        cat = ['condition','participantID']
        pre = ColumnTransformer([
            ('num', StandardScaler(), num),
            ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), cat)
        ])

        clf = Pipeline([('pre', pre),
                        ('logit', LogisticRegression(max_iter=200, solver='lbfgs'))])
        clf.fit(Xtr, ytr)
        p = clf.predict_proba(Xte)[:,1]

        try:
            aucs.append(roc_auc_score(yte, p))
        except Exception:
            aucs.append(np.nan)
        lls.append(log_loss(yte, p, labels=[0,1]))
        brs.append(brier_score_loss(yte, p))

    return {'CV_AUC': float(np.nanmean(aucs)),
            'CV_LogLoss': float(np.mean(lls)),
            'CV_Brier': float(np.mean(brs))}

metrics_rows = []
for c in ['SNR_angle','SNR_side_max','SNR_cue_weighted','SNR_cond_ord','SNR_hybrid_emp']:
    try:
        m = cv_metrics_for_snr(df, c)
        m['SNR'] = c
        metrics_rows.append(m)
    except Exception as e:
        metrics_rows.append({'SNR': c, 'CV_AUC': np.nan, 'CV_LogLoss': np.nan, 'CV_Brier': np.nan})

metrics = pd.DataFrame(metrics_rows).set_index('SNR').sort_values('CV_LogLoss')
print(metrics)
metrics.to_csv(SAVE_DIR / 'cv_metrics.csv', index=True)


## 9. Calibration (reliability curve)

In [ ]:
def reliability_curve(y_true, y_prob, n_bins=10):
    dfc = pd.DataFrame({'y': y_true, 'p': y_prob}).dropna()
    dfc['bin'] = pd.qcut(dfc['p'], q=n_bins, duplicates='drop')
    out = dfc.groupby('bin').agg(p_hat=('p','mean'), y_bar=('y','mean'), n=('y','size')).reset_index()
    return out

def plot_reliability_curve(out: pd.DataFrame):
    plt.figure()
    plt.plot(out['p_hat'], out['y_bar'], marker='o')
    plt.plot([0,1],[0,1], linestyle='--')
    plt.xlabel('Predicted accuracy')
    plt.ylabel('Observed accuracy')
    plt.title('Reliability curve')
    plt.tight_layout()
    plt.show()

# Example: choose best by LogLoss and plot reliability
best = metrics.sort_values('CV_LogLoss').index[0]
print('Best by LogLoss:', best)

X = df[[best,'valid_cue']].copy()
X['condition'] = df['condition']
X['participantID'] = df['participantID']
y = df['correct'].astype(int).values

num = [best, 'valid_cue']
cat = ['condition','participantID']
pre = ColumnTransformer([('num', StandardScaler(), num),
                         ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), cat)])
clf = Pipeline([('pre', pre), ('logit', LogisticRegression(max_iter=200, solver='lbfgs'))])
clf.fit(X, y)
p_full = clf.predict_proba(X)[:,1]

rel = reliability_curve(y, p_full, n_bins=10)
print(rel)
plot_reliability_curve(rel)
rel.to_csv(SAVE_DIR / f'reliability_{best}.csv', index=False)


## 10. Plots

In [ ]:
for c in ['SNR_angle','SNR_side_max','SNR_cue_weighted','SNR_cond_ord','SNR_hybrid_emp']:
    try:
        plot_accuracy_vs_snr(df, c, n_bins=5)
        plot_density_tp(df, c)
    except Exception as e:
        print('Plotting issue for', c, '->', e)


## 11. Export: Winner SNR recipe (for pipeline)

In [ ]:
winner = metrics.sort_values('CV_LogLoss').index[0]
recipe = (
    'Recommended SNR: ' + winner + '\n\n'
    'Computation:\n'
    '1) Estimate sigma per participant×side from TP=0 trials using MAD×1.4826 (fallback to std if MAD=0).\n'
    '2) For each trial, compute:\n'
    '   - target_tilt_abs: if TP=1, absolute tilt of the actual target side (cue side if valid, opposite if invalid);\n'
    '                      if TP=0, 0 (for angle-based) or max(|line1|,|line2|) (for side-max).\n'
    '3) SNR candidates:\n'
    '   - SNR_angle = target_tilt_abs / sigma_target_side\n'
    '   - SNR_side_max = max(|line1|,|line2|) / mean(sigma1, sigma2)\n'
    '   - SNR_cue_weighted = w * tilt_cued / sigma_cued, with w=1 if valid_cue else 0.5\n'
    "   - SNR_cond_ord = map {'50_50':1, '80_20':2, '100_0':3}\n"
    '   - SNR_hybrid_emp = d′(participant×condition) assigned to trials\n'
    '4) (Optional) z-score SNR within participant.\n'
)
path = Path('outputs/snr_eval')
path.mkdir(parents=True, exist_ok=True)
(path / 'winner_snr_recipe.txt').write_text(recipe, encoding='utf-8')
print(recipe)
print('Saved to:', path / 'winner_snr_recipe.txt')
